In [1]:
import pickle

questions = ["Is Public Enemy No. 1 - Todestrieb a crime thriller?",
             "Did Christopher Nolan direct Inception?",
             "Is GoldenEye 007 a James Bond movie?",
             "Is Following a black and white film?",
             "Does the The Lord of the Rings trilogy consist of three movies?",
             "Does First Man depict the life of Neil Armstrong?",
             "Is La Princesse de Clèves set in the French Renaissance period?",
             "Is 2001: A Space Odyssey directed by Stanley Kubrick?",
             "Is Devil in the Flesh 2  a sequel?",
             "Did James Cameron direct Titanic?",
             "Is The Godfather based on a novel by Mario Puzo?",
             "Is The Masked Gang: Cyprus a Turkish comedy film?",
             "Does Star Wars: Episode VI - Return of the Jedi feature the character Luke Skywalker?",
             "Who directed the movie Apocalypse Now?",
             "What movie won the Best Picture Oscar in 2021?",
             "Which film features a character named Sarah Connor?",
             "Can you list three movies directed by Martin Scorsese?",
             "What is the highest-grossing film of all time?",
             "Who composed the score for the 'Pirates of the Caribbean' series?",
             "Name a science fiction movie with a twist ending.",
             "What's the name of the fictional land in 'The Lord of the Rings'?",
             "Who played the role of Harry Potter in the movie series?",
             "What is the name of the ship in 'Titanic'?",
             "In which movie did Tom Hanks say, 'Life is like a box of chocolates'?",
             "Which actor has played the role of Spider-Man on film?",
             "Name a movie where Meryl Streep plays a chef.",
             "Who starred as the leading actor in 'The Revenant'?",
             "Can you list three films Jennifer Lawrence has been in?",
             "What was Will Smith's first movie?",
             "Who directed 'The Grand Budapest Hotel'?",
             "Name a movie directed by Ava DuVernay.",
             "Which director is known for the 'Dark Knight' trilogy?",
             "Who directed 'Schindler's List' and what year was it released?",
             "Can you name a horror movie directed by Jordan Peele?",
             "What is the IMDb rating for 'The Shawshank Redemption'?",
             "Which movie has a perfect score on Rotten Tomatoes?",
             "Can you give me the Metacritic score for 'Mad Max: Fury Road'?",
             "What is the parental rating for 'Jurassic Park'?",
             "How many stars did 'La La Land' receive in its original New York Times review?",
             "Can you show me a positive review for 'The Godfather'?",
             "What did critics say about 'Avatar'?",
             "Are there any Oscar-winning movies that received bad reviews?",
             "What was the general consensus on 'The Shape of Water' after its release?",
             "Did 'Black Panther' get good reviews?",
             "What 1994 crime film revitalized John Travolta's career?",
             "Which movie's famous line is 'I'll be back'?",
             "What is the fictional brand of cigarettes in Quentin Tarantino's films?",
             "Can you name a movie where the protagonist is a toy?",
             "Which film features an iconic dance scene between Uma Thurman and John Travolta?",
             "Which movies are part of the Marvel Cinematic Universe Phase 4?",
             "Name a movie that has been remade more than once.",
             "What is the theme song of 'Mission: Impossible'?",
             "Who voiced Woody in 'Toy Story'?",
             "What are the three rules for caring for a Mogwai in 'Gremlins'?"
             ]


In [2]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer


def get_entities(sentence):
    # Load pre-trained model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
    model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

    # Tokenize the sentence and obtain model outputs
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs).logits
    predictions = torch.argmax(outputs, dim=2)[0].tolist()

    # Convert token and label IDs to strings
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    labels = [model.config.id2label[label_id] for label_id in predictions]

    # Collect entities and their labels
    m_entities = []
    p_entities = []
    entity = ""
    current_label = None  # Keep track of the current entity label
    for token, label in zip(tokens, labels):
        if label.startswith("B") or label.startswith("I"):  # Beginning or Inside of an entity
            if token.startswith("##"):  # Continuation of a word
                entity += token[2:]  # Remove subword prefix
            else:  # New word
                entity += " " + token  # Add space before new word
            current_label = label  # Update current entity label
        elif entity:  # Outside of an entity, but entity string is non-empty
            entity = entity.strip()  # Remove trailing space
            if current_label == "I-PER":
                p_entities.append(entity)
            elif current_label == "I-MISC":
                m_entities.append(entity)
            entity = ""  # Reset entity string
            current_label = None  # Reset current entity label

    # If sentence ends with an entity, append it to the appropriate list
    if entity:
        entity = entity.strip()
        if current_label == "I-PER":
            p_entities.append(entity)
        elif current_label == "I-MISC":
            m_entities.append(entity)

    return p_entities, m_entities


# Example usage:
sentence = "Leonardo DiCaprio starred in Titanic."
p_entities, m_entities = get_entities(sentence)
print("People:", p_entities)
print("Movies:", m_entities)


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


People: ['Leonardo DiCaprio']
Movies: ['Titanic']


In [3]:
print(get_entities(questions[1]))

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(['Christopher Nolan'], ['Inception'])


In [4]:
with open("../dataset/film_dict.pickle", 'rb') as f:
    film_dict = pickle.load(f)
movie_titles = list(film_dict.keys())

with open("human_dict.pickle", 'rb') as f:
    human_dict = pickle.load(f)
human_dict = list(human_dict.keys())

In [8]:
import pickle
from rapidfuzz import fuzz, process


def get_movie_title(query):
    # Extract all movie titles from your dictionary

    # Use fuzzywuzzy to find the closest match in your dictionary to the user query
    best_match = process.extract(query, movie_titles, scorer=fuzz.ratio, limit=2)

    # best_match is a tuple containing the best matching movie title and a score
    # matching_movie_title, score = best_match
    matched_movies = []
    for movie in best_match:
        print("FUZZYWUZZY results:", movie)

        if int(movie[1]) > 50:
            matched_movies.append(movie[0])
    return matched_movies


In [10]:
for question in questions:
    p_entities, m_entities = get_entities(question)
    print("_____________________")
    print("Question: ", question)
    print("People:", p_entities)
    print("Movies:", m_entities)
    print("FUZZ:",get_movie_title(''.join(m_entities)))

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


_____________________
Question:  Is Public Enemy No. 1 - Todestrieb a crime thriller?
People: []
Movies: ['Public Enemy No . 1', 'Todestrieb']
FUZZYWUZZY results: ('public enemy no. 1 – todestrieb', 80.0, 7309)
FUZZYWUZZY results: ('license to drive', 53.333333333333336, 11834)
FUZZ: ['public enemy no. 1 – todestrieb', 'license to drive']


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


_____________________
Question:  Did Christopher Nolan direct Inception?
People: ['Christopher Nolan']
Movies: ['Inception']
FUZZYWUZZY results: ('inception', 88.88888888888889, 22422)
FUZZYWUZZY results: ('conception', 84.21052631578947, 10051)
FUZZ: ['inception', 'conception']


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

_____________________
Question:  What is the theme song of 'Mission: Impossible'?
People: []
Movies: ['Mission : Impossible']
FUZZYWUZZY results: ('mission: impossible', 87.17948717948718, 21973)
FUZZYWUZZY results: ('mission: impossible 2', 82.92682926829268, 22186)
FUZZ: ['mission: impossible', 'mission: impossible 2']


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


_____________________
Question:  Who voiced Woody in 'Toy Story'?
People: ['Woody']
Movies: ['Toy Story']
FUZZYWUZZY results: ('toy story', 77.77777777777779, 24333)
FUZZYWUZZY results: ('tokyo story', 70.0, 18086)
FUZZ: ['toy story', 'tokyo story']


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


_____________________
Question:  What are the three rules for caring for a Mogwai in 'Gremlins'?
People: []
Movies: ['Gremlins']
FUZZYWUZZY results: ('gremlins', 87.5, 23900)
FUZZYWUZZY results: ('redline', 66.66666666666667, 17991)
FUZZ: ['gremlins', 'redline']


In [11]:
print(movie_titles)

['jan dara', 'moondram pirai', "buffalo bill and the indians, or sitting bull's history lesson", 'what we wanted', 'wanted: dead or alive', 'linger', 'eastern condors', 'amerika', 'bukowski: born into this', 'fatal move', 'on the mountain of tai hang', 'forever enthralled', 'kill the irishman', 'bullseye!', 'bully', 'dry wind', 'the big hit', 'angels fallen', 'gagarine', 'golimaar', 'adhurs', 'rose island', 'simha', 'rakta charitra', 'khaleja', 'gorillas in the mist', 'operation christmas drop', 'jasmine women', 'blind mountain', 'turn left, turn right', 'burke & hare', 'burma vj', 'burning bright', 'butter', 'population boom', 'bye bye love', 'bye bye blackbird', 'bye bye birdie', 'bye bye bluebird', 'the chinese feast', 'stage door', 'the truth about jane and sam', 'the longest nite', 'naked killer', 'c.i.d.', 'c.r.a.z.y.', 'the super', 'notes for my son', 'csny/déjà vu', 'cash', 'nightbreed', 'kaos', 'cadillac man', 'caged', 'cairo time', 'daisy diamond', 'the baytown outlaws', 'the